### validation 결과 확인

In [28]:
import os
import numpy as np
import pandas as pd

targets = pd.DataFrame(columns=['target', 'total', 'train', 'valid', 'f1'])
targets['target'] = np.arange(7)

# path = '../../v1.3.1_aug3'
path = '../../v1.3.2'

total = pd.read_csv(os.path.join(path, 'train.csv'))
val_pred = pd.read_csv(os.path.join(path, 'valid_output.csv'))

val_ids = list(val_pred['ID'].values)
train = total[~total['ID'].str.contains('|'.join(val_ids))]
val_answer = total[total['ID'].str.contains('|'.join(val_ids))]

label = range(7)
for l in label:
    targets.loc[targets['target']==l, 'total'] = len(total[total['target']==l])
    targets.loc[targets['target']==l, 'train'] = len(train[train['target']==l])
    targets.loc[targets['target']==l, 'valid'] = len(val_pred[val_pred['target']==l])

    val_correct = val_pred[(val_pred['target']==l) & val_pred['ID'].str.contains('|'.join((val_answer[val_answer['target']==l]['ID'].values)))]
    recall = len(val_correct) / (len(val_answer[val_answer['target']==l]))
    precision = len(val_correct) / (len(val_pred[val_pred['target']==l]))

    targets.loc[targets['target']==l, 'f1'] = 2 * precision * recall / (precision + recall)

targets

,target,total,train,valid,f1
0,0,336,258,77,0.787097
1,1,365,289,68,0.902778
2,2,309,248,52,0.79646
3,3,248,176,48,0.666667
4,4,322,262,56,0.896552
5,5,329,270,62,0.826446
6,6,358,269,91,0.888889


### LLaMA~♡

In [1]:
import json
import pandas as pd

with open('key_maps.json', 'r', encoding='utf-8') as f:
    key_maps = json.load(f)

train = pd.read_csv('../../v1.3.0/train.csv')
train.sample(5)

,ID,text,target,polluted_lv
1329,ynat-v1_train_02785,민주화운동 보상받으면 국가배상 안 돼헌재와 반대 판결,2,0.000000
1172,ynat-v1_train_02444,가상화폐 인기에 악성코드 기승랜섬웨어 신고 37배,4,0.074074
30,ynat-v1_train_00073,대한항공 인천바르셀로나 노선 첫 취항,5,0.000000
368,ynat-v1_train_00776,민주 평창올림픽 성공 개최 총력전색깔론 한국당 맹공,2,0.000000
1058,ynat-v1_train_02187,주말 N 여행 강원권 쫀득한 찰옥수수일편단심 무궁화,0,0.035714


In [63]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B' # 'beomi/Llama-3-Open-Ko-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto'
)

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): L

In [22]:
import pandas as pd
from tqdm.notebook import tqdm

train_fewshot = pd.read_csv('../../v1.3.0/train.csv') # text drop, cleanlab, no aug

generated = pd.DataFrame(columns=['ID', 'text', 'target', 'polluted_lv'])
for i in tqdm(range(3, 4), total=1, position=0):
    key = key_maps[str(i)]
    new_aug = pd.DataFrame(columns=['ID', 'text', 'target', 'polluted_lv'])

    PROMPT = f'''당신은 '{key}' 분야의 기사 제목을 작성하는 전문 작가입니다. 주어진 예시를 바탕으로 다양한 형태의 기사 제목을 작성해 주세요.'''

    shots = train_fewshot[(train_fewshot['target']==i)].sample(10, random_state=42)['text'].values
    fewshot = []
    for shot in shots:
        fewshot.append({"role": "user", "content": f"'{key}' 분야에 해당하는 기사 제목을 한 개만 생성하세요."})
        fewshot.append({"role": "assistant", "content": shot})
    # fewshot = [{"role": "assistant", "content": "\n".join(shots)}]

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    cnt = 0
    for j in tqdm(range(100), desc='labeling', total=100, position=1, leave=False):
        # messages = [{"role": "system", "content": PROMPT}] + fewshot + [{"role": "user", "content": f"'{key}' 분야에 해당하는 기사 제목을 생성하세요."}]
        messages = [{"role": "system", "content": PROMPT}] + fewshot + [{"role": "user", "content": f"'{key}' 분야에 해당하는 기사 제목을 한 개만 생성하세요."}]
        
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors='pt'
        ).to(model.device)

        outputs = model.generate(
            input_ids,
            max_new_tokens=512,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9
        )

        result = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        new_aug.loc[j] = [f'augmented-{i}-{j}', result, i, 0]
        # print(result)

    generated = pd.concat([generated, new_aug])

len(generated)
generated.to_csv('../../v1.3.2/augmented.csv', index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

labeling:   0%|          | 0/100 [00:00<?, ?it/s]

### Back Translation

In [6]:
from transformers import pipeline

translator_1 = pipeline('translation', model='NHNDQ/nllb-finetuned-en2ko', device=0, src_lang='kor_Hang', tgt_lang='eng_Latn', max_length=512)
translator_2 = pipeline('translation', model='NHNDQ/nllb-finetuned-en2ko', device=0, src_lang='eng_Latn', tgt_lang='kor_Hang', max_length=512)

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

data = pd.read_csv('../../v1.3.0/train.csv')

rtt = pd.DataFrame(columns=['ID', 'text', 'target', 'polluted_lv'])
for i in tqdm(range(7), desc='rtt', total=7, position=0):
    target = data[(data['target']==i) & (data['polluted_lv']<=0.10)]['text'].values

    rtt_tmp = pd.DataFrame(columns=['ID', 'text', 'target', 'polluted_lv'])
    for j, text in tqdm(enumerate(target), desc=f'{i}', total=len(target), position=1, leave=False):
        # print(text)
        output = translator_1(text, max_length=512ㄴ)
        # print(output[0]['translation_text'])
        output = translator_2(output[0]['translation_text'], max_length=512)
        # print(output[0]['translation_text'])
        # print()
        rtt_tmp.loc[j] = [f'rtt-{i}-{j}', output[0]['translation_text'], i, 0]
    rtt = pd.concat([rtt, rtt_tmp])
print(len(rtt))
rtt.to_csv('../../v1.3.2/rtt_2_50.csv', index=False)

In [27]:
import pandas as pd

v131 = pd.read_csv('../../v1.3.1_aug3/train.csv')
rtt = pd.read_csv('../../v1.3.2/rtt_2.csv')
rtt['target'].value_counts()

rttt = pd.DataFrame(columns=['ID', 'text', 'target', 'polluted_lv'])
for i in range(7):
    if i==3:
        rttt = pd.concat([rttt, rtt[rtt['target']==i]])    
    else:
        rttt = pd.concat([rttt, rtt[rtt['target']==i].sample(50)])
len(rttt)

v132 = pd.concat([v131, rttt])
v132.to_csv('../../v1.3.2/train.csv', index=False)
len(v132)

2267

### EDA
- v1.3.2, label=3에 SR 시도해보기

In [ ]:
import pandas as pd



### Mix-Up